In [ ]:
import pandas as pd
import random
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install -U spacy
!python -m spacy download en
import spacy
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-21 14:08:47.607427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enab

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!gdown --id 10fHsgc9E1n_JgaV-mmS818cF9QL_5Wwg # train data from contest1 file
!gdown --id 1gbC4rWAwyBL9ZDHnQRqPbKOhPBPAlh5z  # evaluate.py
!gdown --id 1KAj7PDbRq48kyZBNg_Efi1Gn-kfHSI84 # test data

In [ ]:
#1) Prepare Data

data = pd.read_csv('contest1_train.csv')
data

,id,text,aspectCategory,polarity
0,3121,But the staff was so horrible to us.,service,negative
1,2777,"To be completely fair, the only redeeming fact...",food,positive
2,2777,"To be completely fair, the only redeeming fact...",anecdotes/miscellaneous,negative
3,1634,"The food is uniformly exceptional, with a very...",food,positive
4,2534,Where Gabriela personaly greets you and recomm...,service,positive
...,...,...,...,...
3151,2853,I would highly recommend this place to everyon...,anecdotes/miscellaneous,positive
3152,1163,The service is fantastic.,service,positive
3153,216,I recommend that you try this hidden spot whic...,anecdotes/miscellaneous,positive
3154,1109,The garlic mashed potatoes are hands down the ...,food,positive


In [ ]:
test = pd.read_csv('contest1_test.csv')
test

,id,text
0,899,Food and service was okay.
1,1349,"great for a romantic evening, or a fun evening..."
2,934,The lamb meat was under-cooked and EXTRMELY CH...
3,2199,The best pad thai i've ever had.
4,188,"Over time, the food quality has decreased subs..."
...,...,...
456,1063,But that is highly forgivable.
457,777,"From the appetizers we ate, the dim sum and ot..."
458,875,"When we arrived at 6:00 PM, the restaurant was..."
459,671,Each table has a pot of boiling water sunken i...


In [ ]:
data.polarity.value_counts()

positive    1876
negative     715
neutral      398
conflict     167
Name: polarity, dtype: int64

In [ ]:
data.aspectCategory.value_counts()

food                       1051
anecdotes/miscellaneous     956
service                     506
ambience                    368
price                       275
Name: aspectCategory, dtype: int64

In [ ]:
import re
from sklearn.feature_extraction import DictVectorizer
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
nltk.download('stopwords')
def tokenize(text):
  tokens = nltk.word_tokenize(text)
  patt = re.compile('[a-zA-Z0-9]+')
  new_tokens = [x.lower() for x in tokens if patt.match(x) and not x in all_stopwords]
  return '|'.join(new_tokens)
data['tokens'] = data.text.apply(tokenize)
clean_data = data[['id', 'tokens', 'aspectCategory', 'polarity']]
clean_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,id,tokens,aspectCategory,polarity
0,3121,but|staff|horrible,service,negative
1,2777,to|completely|fair|redeeming|factor|food|avera...,food,positive
2,2777,to|completely|fair|redeeming|factor|food|avera...,anecdotes/miscellaneous,negative
3,1634,the|food|uniformly|exceptional|capable|kitchen...,food,positive
4,2534,where|gabriela|personaly|greets|recommends|eat,service,positive
...,...,...,...,...
3151,2853,i|highly|recommend|place|occasion,anecdotes/miscellaneous,positive
3152,1163,the|service|fantastic,service,positive
3153,216,i|recommend|try|hidden|spot|surely|blowup|soon,anecdotes/miscellaneous,positive
3154,1109,the|garlic|mashed|potatoes|hands|best|city,food,positive


In [ ]:
test = pd.read_csv('contest1_test.csv')
test['tokens'] = test.text.apply(tokenize)
test = test[['id', 'tokens']]

# SERVICE_ASPECT

In [ ]:
def get_service_aspect(df):
  df['service_aspect'] = 0
  #Set the value of the new column to 1 if the existing column equals "service"
  df.loc[df['aspectCategory'] == 'service', 'service_aspect'] = 1
   # Group the DataFrame by id and check for duplicates where one row has service_column=1 and another has service_column=0
  groups = df.groupby('id')
  for group_name, group_df in groups:
      if (group_df['service_aspect'] == 1).any() and (group_df['service_aspect'] == 0).any():
          df = df.drop(group_df.loc[group_df['service_aspect'] == 0].index)
  return df

service_data = get_service_aspect(clean_data)
service_data

,id,tokens,aspectCategory,polarity,service_aspect
0,3121,But|staff|horrible,service,negative,1
1,2777,To|completely|fair|redeeming|factor|food|avera...,food,positive,0
2,2777,To|completely|fair|redeeming|factor|food|avera...,anecdotes/miscellaneous,negative,0
3,1634,The|food|uniformly|exceptional|capable|kitchen...,food,positive,0
4,2534,Where|Gabriela|personaly|greets|recommends|eat,service,positive,1
...,...,...,...,...,...
3151,2853,I|highly|recommend|place|occasion,anecdotes/miscellaneous,positive,0
3152,1163,The|service|fantastic,service,positive,1
3153,216,I|recommend|try|hidden|spot|surely|blowup|soon,anecdotes/miscellaneous,positive,0
3154,1109,The|garlic|mashed|potatoes|hands|best|city,food,positive,0


In [ ]:
# Prepare data
train, dev = train_test_split(service_data, train_size=0.8, random_state=100)
# Feature engineering
def featurize(text):
    return {x:1 for x in text.split('|')}

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

# Train model
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['service_aspect'])

# Evaluation
dev_features = dev['tokens'].apply(featurize)
dev_feature_vectors = vectorizer.transform(dev_features)
dev_predictions = lr_text_classifier.predict(dev_feature_vectors)
print (classification_report(dev_predictions, dev['service_aspect']))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90       280
           1       0.34      0.56      0.42        36

    accuracy                           0.83       316
   macro avg       0.64      0.71      0.66       316
weighted avg       0.87      0.83      0.84       316



In [ ]:
test = pd.read_csv('contest1_test.csv')
test['tokens'] = test.text.apply(tokenize)
test = test[['id', 'tokens']]

In [ ]:
test_features = test['tokens'].apply(featurize)
test_feature_vectors = vectorizer.transform(test_features)
test_pred_service_aspect = lr_text_classifier.predict(test_feature_vectors)

#print (classification_report(test_predictions, test['']))
def map_prediction(df, test_pred_array, label_name):
    df[label_name] = test_pred_array
    df = df[df[label_name] != 0]
    df['aspectCategory'] = np.where(df[label_name] != 0, label_name, None)
    return df
service_pred_test = map_prediction(test, test_pred_service_aspect, 'service')
service_pred_test

<ipython-input-42-d6cdeacad079>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aspectCategory'] = np.where(df[label_name] != 0, label_name, None)


,id,tokens,service,aspectCategory
4,188,Over|time|food|quality|decreased|substantially...,1,service
5,1748,Great|pizza|fantastic|service,1,service
19,1030,As|far|service|goes|waitresses|particularly|fr...,1,service
41,817,The|waiter|attentive,1,service
53,2263,We|went|9:30|Friday|died|bit|service|great,1,service
60,1496,Turned|service|upstairs|sat,1,service
80,2779,My|host|asked|appetizers|group|waiter|gave|ord...,1,service
85,3341,During|course|past|3|months|chef|staff|changed...,1,service
98,3666,Yes|wait|seated|small|waiting|area|seat|bar|taken,1,service
102,2051,Ive|times|waiters|know,1,service


# FOOD_ASPECT

In [ ]:
def get_food_aspect(df):
  df['food_aspect'] = 0
  # Set the value of the new column to 1 if the existing column equals "food"
  df.loc[df['aspectCategory'] == 'food', 'food_aspect'] = 1
  groups = df.groupby('id')
  for group_name, group_df in groups:
      if (group_df['food_aspect'] == 1).any() and (group_df['food_aspect'] == 0).any():
          df = df.drop(group_df.loc[group_df['food_aspect'] == 0].index)

  # Drop the temporary service_column
  return df

food_aspect = get_food_aspect(clean_data)
food_aspect

,id,tokens,aspectCategory,polarity,food_aspect
0,3121,But|staff|horrible,service,negative,0
1,2777,To|completely|fair|redeeming|factor|food|avera...,food,positive,1
3,1634,The|food|uniformly|exceptional|capable|kitchen...,food,positive,1
4,2534,Where|Gabriela|personaly|greets|recommends|eat,service,positive,0
5,583,For|enjoy|I,anecdotes/miscellaneous,positive,0
...,...,...,...,...,...
3151,2853,I|highly|recommend|place|occasion,anecdotes/miscellaneous,positive,0
3152,1163,The|service|fantastic,service,positive,0
3153,216,I|recommend|try|hidden|spot|surely|blowup|soon,anecdotes/miscellaneous,positive,0
3154,1109,The|garlic|mashed|potatoes|hands|best|city,food,positive,1


In [ ]:
# Prepare data
train, dev = train_test_split(food_aspect, train_size=0.8, random_state=100)
#SERVICE ASPECT

# Feature engineering
def featurize(text):
    return {x:1 for x in text.split('|')}

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

# Train model
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['food_aspect'])

# Evaluation
dev_features = dev['tokens'].apply(featurize)
dev_feature_vectors = vectorizer.transform(dev_features)
dev_predictions = lr_text_classifier.predict(dev_feature_vectors)
print (classification_report(dev_predictions, dev['food_aspect']))

              precision    recall  f1-score   support

           0       0.92      0.81      0.86       174
           1       0.68      0.86      0.76        83

    accuracy                           0.82       257
   macro avg       0.80      0.83      0.81       257
weighted avg       0.84      0.82      0.83       257



In [ ]:
test_features = test['tokens'].apply(featurize)
test_feature_vectors = vectorizer.transform(test_features)
test_pred_food_aspect = lr_text_classifier.predict(test_feature_vectors)
food_pred_test = map_prediction(test, test_pred_food_aspect, 'food')
food_pred_test

<ipython-input-42-d6cdeacad079>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aspectCategory'] = np.where(df[label_name] != 0, label_name, None)


,id,tokens,food,aspectCategory
0,899,Food|service|okay,1,food
2,934,The|lamb|meat|under-cooked|EXTRMELY|CHEWY,1,food
3,2199,The|best|pad|thai,1,food
4,188,Over|time|food|quality|decreased|substantially...,1,food
5,1748,Great|pizza|fantastic|service,1,food
...,...,...,...,...
420,2325,The|selection|changes|frequently|basic|dishes|...,1,food
436,2116,ordered|dinosaur|rolls|white|tuna|sashimi,1,food
454,1027,We|shared|bottle|sake|order|edamames|sushi|pla...,1,food
457,777,From|appetizers|ate|dim|sum|variety|foods|impo...,1,food


# ANECDOTES_ASPECT

In [ ]:
def get_anecdotes_aspect(df):
  df['anecdotes_aspect'] = 0
  df.loc[df['aspectCategory'] == 'anecdotes/miscellaneous', 'anecdotes_aspect'] = 1
  groups = df.groupby('id')
  for group_name, group_df in groups:
      if (group_df['anecdotes_aspect'] == 1).any() and (group_df['anecdotes_aspect'] == 0).any():
          df = df.drop(group_df.loc[group_df['anecdotes_aspect'] == 0].index)
  # Drop the temporary service_column
  return df

anecdotes_aspect = get_anecdotes_aspect(clean_data)
anecdotes_aspect

,id,tokens,aspectCategory,polarity,anecdotes_aspect
0,3121,But|staff|horrible,service,negative,0
2,2777,To|completely|fair|redeeming|factor|food|avera...,anecdotes/miscellaneous,negative,1
3,1634,The|food|uniformly|exceptional|capable|kitchen...,food,positive,0
4,2534,Where|Gabriela|personaly|greets|recommends|eat,service,positive,0
5,583,For|enjoy|I,anecdotes/miscellaneous,positive,1
...,...,...,...,...,...
3151,2853,I|highly|recommend|place|occasion,anecdotes/miscellaneous,positive,1
3152,1163,The|service|fantastic,service,positive,0
3153,216,I|recommend|try|hidden|spot|surely|blowup|soon,anecdotes/miscellaneous,positive,1
3154,1109,The|garlic|mashed|potatoes|hands|best|city,food,positive,0


In [ ]:
# Prepare data
train, dev = train_test_split(anecdotes_aspect, train_size=0.8, random_state=100)

#anecdotes_aspect

# Feature engineering
def featurize(text):
    return {x:1 for x in text.split('|')}

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

# Train model
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['anecdotes_aspect'])

# Evaluation
dev_features = dev['tokens'].apply(featurize)
dev_feature_vectors = vectorizer.transform(dev_features)
dev_predictions = lr_text_classifier.predict(dev_feature_vectors)
print (classification_report(dev_predictions, dev['anecdotes_aspect']))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       429
           1       0.71      0.82      0.76       175

    accuracy                           0.85       604
   macro avg       0.82      0.84      0.83       604
weighted avg       0.86      0.85      0.85       604



In [ ]:
test_features = test['tokens'].apply(featurize)
test_feature_vectors = vectorizer.transform(test_features)
test_pred_anecdotes_aspect = lr_text_classifier.predict(test_feature_vectors)

anecdotes_pred_test = map_prediction(test, test_pred_anecdotes_aspect, 'anecdotes/miscellaneous')
anecdotes_pred_test

<ipython-input-16-d6cdeacad079>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aspectCategory'] = np.where(df[label_name] != 0, label_name, None)


,id,tokens,anecdotes/miscellaneous,aspectCategory
7,390,ask|oil|time,1,anecdotes/miscellaneous
9,1428,I|went|lunch|good|I|expected|reviews|I|read,1,anecdotes/miscellaneous
10,826,I|know|expect|restaurant|literally|overlooks|m...,1,anecdotes/miscellaneous
15,190,I|recommend|place,1,anecdotes/miscellaneous
18,1811,know|busy|showing|vintage|tee|shirt|looking|bo...,1,anecdotes/miscellaneous
...,...,...,...,...
451,3168,Better|bagel|shop|corner|worth|going|way|opinion,1,anecdotes/miscellaneous
455,1735,I|believe|people|complain|cheese|sticks,1,anecdotes/miscellaneous
456,1063,But|highly|forgivable,1,anecdotes/miscellaneous
458,875,When|arrived|6:00|PM|restaurant|practically,1,anecdotes/miscellaneous


# AMBIENCE_ASPECT

In [ ]:
def get_ambience_aspect(df):
  df['ambience_aspect'] = 0
  df.loc[df['aspectCategory'] == 'ambience', 'ambience_aspect'] = 1
  groups = df.groupby('id')
  for group_name, group_df in groups:
      if (group_df['ambience_aspect'] == 1).any() and (group_df['ambience_aspect'] == 0).any():
          df = df.drop(group_df.loc[group_df['ambience_aspect'] == 0].index)
  return df

ambience_aspect = get_ambience_aspect(clean_data)
ambience_aspect

,id,tokens,aspectCategory,polarity,ambience_aspect
0,3121,But|staff|horrible,service,negative,0
1,2777,To|completely|fair|redeeming|factor|food|avera...,food,positive,0
2,2777,To|completely|fair|redeeming|factor|food|avera...,anecdotes/miscellaneous,negative,0
3,1634,The|food|uniformly|exceptional|capable|kitchen...,food,positive,0
4,2534,Where|Gabriela|personaly|greets|recommends|eat,service,positive,0
...,...,...,...,...,...
3151,2853,I|highly|recommend|place|occasion,anecdotes/miscellaneous,positive,0
3152,1163,The|service|fantastic,service,positive,0
3153,216,I|recommend|try|hidden|spot|surely|blowup|soon,anecdotes/miscellaneous,positive,0
3154,1109,The|garlic|mashed|potatoes|hands|best|city,food,positive,0


In [ ]:
train, dev = train_test_split(ambience_aspect, train_size=0.8, random_state=100)

#anecdotes_aspect

# Feature engineering
def featurize(text):
    return {x:1 for x in text.split('|')}

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

# Train model
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['ambience_aspect'])

# Evaluation
dev_features = dev['tokens'].apply(featurize)
dev_feature_vectors = vectorizer.transform(dev_features)
dev_predictions = lr_text_classifier.predict(dev_feature_vectors)
print (classification_report(dev_predictions, dev['ambience_aspect']))

              precision    recall  f1-score   support

           0       0.99      0.92      0.95       555
           1       0.37      0.90      0.53        31

    accuracy                           0.91       586
   macro avg       0.68      0.91      0.74       586
weighted avg       0.96      0.91      0.93       586



In [ ]:
test_features = test['tokens'].apply(featurize)
test_feature_vectors = vectorizer.transform(test_features)
test_pred_ambience_aspect = lr_text_classifier.predict(test_feature_vectors)
ambience_pred_test = map_prediction(test, test_pred_ambience_aspect, 'ambience')
ambience_pred_test

<ipython-input-16-d6cdeacad079>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aspectCategory'] = np.where(df[label_name] != 0, label_name, None)


,id,tokens,ambience,aspectCategory
8,885,While|pretty|place|overly|cute|French|way|food...,1,ambience
27,468,Restaurant|snobs|need|bother|small|neighborhoo...,1,ambience
52,3520,There|actually|space|breathe|decor|sets|tone|i...,1,ambience
84,2580,If|food|A+++|service|ambience,1,ambience
86,722,I|like|ambience|dark|original,1,ambience
95,1676,You|paying|atmosphere|nice|numerous|places|Bay...,1,ambience
103,862,The|decor|simple|comfortable,1,ambience
122,2146,The|staff|makes|feel|home|food|great|atmospher...,1,ambience
147,1718,Great|food|great|decor|great|service,1,ambience
162,2686,The|place|small|intimate|feel|little|crowded|s...,1,ambience


# PRICE_ASPECT

In [ ]:
def get_price_aspect(df):
  df['price_aspect'] = 0
  df.loc[df['aspectCategory'] == 'price', 'price_aspect'] = 1
  groups = df.groupby('id')
  for group_name, group_df in groups:
      if (group_df['price_aspect'] == 1).any() and (group_df['price_aspect'] == 0).any():
          df = df.drop(group_df.loc[group_df['price_aspect'] == 0].index)
  return df

price_aspect = get_price_aspect(clean_data)
price_aspect

,id,tokens,aspectCategory,polarity,price_aspect
0,3121,But|staff|horrible,service,negative,0
1,2777,To|completely|fair|redeeming|factor|food|avera...,food,positive,0
2,2777,To|completely|fair|redeeming|factor|food|avera...,anecdotes/miscellaneous,negative,0
3,1634,The|food|uniformly|exceptional|capable|kitchen...,food,positive,0
4,2534,Where|Gabriela|personaly|greets|recommends|eat,service,positive,0
...,...,...,...,...,...
3151,2853,I|highly|recommend|place|occasion,anecdotes/miscellaneous,positive,0
3152,1163,The|service|fantastic,service,positive,0
3153,216,I|recommend|try|hidden|spot|surely|blowup|soon,anecdotes/miscellaneous,positive,0
3154,1109,The|garlic|mashed|potatoes|hands|best|city,food,positive,0


In [ ]:
# Prepare data
train, dev = train_test_split(price_aspect, train_size=0.8, random_state=100)

#anecdotes_aspect

# Feature engineering
def featurize(text):
    return {x:1 for x in text.split('|')}

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features)

# Train model
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['price_aspect'])

# Evaluation
dev_features = dev['tokens'].apply(featurize)
dev_feature_vectors = vectorizer.transform(dev_features)
dev_predictions = lr_text_classifier.predict(dev_feature_vectors)
print (classification_report(dev_predictions, dev['price_aspect']))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96       565
           1       0.36      1.00      0.53        22

    accuracy                           0.93       587
   macro avg       0.68      0.97      0.75       587
weighted avg       0.98      0.93      0.95       587



In [ ]:
test_features = test['tokens'].apply(featurize)
test_feature_vectors = vectorizer.transform(test_features)
test_pred_price_aspect = lr_text_classifier.predict(test_feature_vectors)
#print (classification_report(test_predictions, test['']))
price_pred_test = map_prediction(test, test_pred_price_aspect, 'price')
price_pred_test

<ipython-input-32-e3b141c88c64>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aspectCategory'] = np.where(df['aspect'] == 1, label_name, None)


,id,tokens,aspectCategory
25,3696,Unlike|places|NYC|sandwiches|want|come|triple-...,price
47,2772,The|tables|crammed|way|close|menu|typical|Ital...,price
57,1195,The|sushi|average|prices,price
83,180,Slightly|pricey|worth,price
90,346,This|place|worth|prices,price
116,1985,Even|relatively|inexpensive|botle|wine|70.00|i...,price
136,1416,Edible|ripoff|prices|given|whats|area,price
200,1060,Spreads|toppings|great|bit|pricey,price
205,1302,And|prices|high|actually|think|away|charging|p...,price
220,983,reasonably|priced|8|dinner|5|lunch|delicious|f...,price


# MAP PREDICTIONS

In [ ]:
ser = {0:0, 1:'service'}
food = {0:0, 1:'food'}
anecdotes = {0:0, 1:'anecdotes/miscellaneous'}
ambience = {0:0, 1:'ambience'}
price ={0:0, 1:'price'}
# Use .replace() to convert the numbers to new values
test['service'] = test['service'].replace(ser)

test['aspectCategory'] = None

test['service'].apply(lambda x: x if x == 'service' else None)
test['food'] = test['food'].replace(food)
test['anecdotes/miscellaneous'] = test['anecdotes/miscellaneous'].replace(anecdotes)
test['ambience'] = test['ambience'].replace(ambience)
test['price'] = test['price'].replace(price)

'''def add_value(df):
  df['aspectCategory'] = df['service'].apply(lambda x: x if x == 'service' else None)
  df['aspectCategory'] = df['food'].apply(lambda x: x if x == 'food' and df['aspectCategory'] is None else None)'''

test['aspectCategory'] = test['food'].apply(lambda x: x if x == 'food' and test['aspectCategory'] is None elif x == 'food' else None)

def check_if_None(x, label):
  if x == label and test['aspectCategory'] is None:

     elif x == 'food' else None)

test

,id,tokens,service,food,anecdotes/miscellaneous,ambience,price,aspectCategory
0,899,Food|service|okay,service,0,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
1,1349,great|romantic|evening|fun|evening|friends,0,0,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
2,934,The|lamb|meat|under-cooked|EXTRMELY|CHEWY,0,food,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
3,2199,The|best|pad|thai,0,food,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
4,188,Over|time|food|quality|decreased|substantially...,service,food,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
...,...,...,...,...,...,...,...,...
456,1063,But|highly|forgivable,0,0,anecdotes/miscellaneous,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
457,777,From|appetizers|ate|dim|sum|variety|foods|impo...,0,food,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
458,875,When|arrived|6:00|PM|restaurant|practically,0,0,anecdotes/miscellaneous,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...
459,671,Each|table|pot|boiling|water|sunken|surface|pl...,0,food,0,0,0,<function <lambda>.<locals>.<lambda> at 0x7f28...


# combine Dataframe

In [ ]:
aspect_pred = pd.concat([service_pred_test, food_pred_test, anecdotes_pred_test, ambience_pred_test, price_pred_test], axis=0)

In [ ]:
aspect_pred = aspect_pred[['id', 'tokens', 'aspectCategory']]

In [ ]:
aspect_pred

,id,tokens,aspectCategory
0,899,Food|service|okay,service
4,188,Over|time|food|quality|decreased|substantially...,service
5,1748,Great|pizza|fantastic|service,service
16,363,great|wine|list|italian|good|food|service|INIT...,service
19,1030,As|far|service|goes|waitresses|particularly|fr...,service
...,...,...,...
330,705,Just|cheap|NOT|mean|portions|small|food|nasty|...,price
336,2226,The|price|extremely|reasonable|appetizers|food...,price
361,151,However|want|great|food|great|price|mind|decor...,price
375,2243,Its|location|good|fact|Hutner|College|near|pri...,price


In [ ]:
c_sorted = aspect_pred.sort_values(by=['id'])
c_sorted
c_sorted.to_csv('aspect_pred_multilabel.csv', index=None)

# TRY BIGRAM

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split data into train and dev sets
train, dev_test = train_test_split(clean_data, train_size=0.8, random_state=100)
dev, test = train_test_split(dev_test, test_size=0.5, random_state=100)
#train, dev_test = np.split(clean_data, [int(len(clean_data)*0.8)]) # train:dev = 80:20
# Define feature engineering function for bigram features
def bigram_featurize(text):
    bigrams = []
    words = text.split('|')
    for i in range(len(words)-1):
        bigrams.append(words[i] + ' ' + words[i+1])
    return {x: 1 for x in bigrams}

# Vectorize train set
train_texts = train['tokens']
train_labels = train['aspectCategory']
train_features = train_texts.apply(bigram_featurize)
vectorizer = DictVectorizer()
train_vectors = vectorizer.fit_transform(train_features)

# Train logistic regression model
lr_classifier = LogisticRegression()
lr_classifier.fit(train_vectors, train_labels)

# Vectorize dev set
dev_texts = dev['tokens']
dev_features = dev_texts.apply(bigram_featurize)
dev_vectors = vectorizer.transform(dev_features)

# Make predictions on dev set
dev_predictions = lr_classifier.predict(dev_vectors)

# Print classification report
print(classification_report(dev_predictions, dev['aspectCategory']))


                         precision    recall  f1-score   support

               ambience       0.05      0.13      0.07        15
anecdotes/miscellaneous       0.89      0.42      0.57       195
                   food       0.24      0.36      0.28        70
                  price       0.00      0.00      0.00         9
                service       0.11      0.22      0.15        27

               accuracy                           0.36       316
              macro avg       0.26      0.23      0.21       316
           weighted avg       0.61      0.36      0.43       316



In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split data into train and dev sets
train, dev = train_test_split(clean_data, train_size=0.8, random_state=100)

# Define feature engineering function for bigram features
def bigram_featurize(text):
    bigrams = []
    words = text.split('|')
    for i in range(len(words)-1):
        bigrams.append(words[i] + ' ' + words[i+1])
    return {x: 1 for x in bigrams}

# Vectorize train set
train_texts = train['tokens']
train_labels = train['polarity']
train_features = train_texts.apply(bigram_featurize)
vectorizer = DictVectorizer()
train_vectors = vectorizer.fit_transform(train_features)

# Train logistic regression model
lr_classifier = LogisticRegression()
lr_classifier.fit(train_vectors, train_labels)

# Vectorize dev set
dev_texts = dev['tokens']
dev_features = dev_texts.apply(bigram_featurize)
dev_vectors = vectorizer.transform(dev_features)

# Make predictions on dev set
dev_predictions = lr_classifier.predict(dev_vectors)

# Print classification report
print(classification_report(dev_predictions, dev['polarity']))

              precision    recall  f1-score   support

    conflict       0.06      0.17      0.09         6
    negative       0.24      0.71      0.36        24
     neutral       0.10      0.57      0.17         7
    positive       0.95      0.64      0.77       279

    accuracy                           0.64       316
   macro avg       0.34      0.52      0.35       316
weighted avg       0.86      0.64      0.71       316

